In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

In [0]:
def process_despesas_silver():
    """
    Processa os dados da tabela tb_despesas_bronze e salva na camada Silver.
    """
    try:
        # Carrega a tabela bronze
        df_bronze = spark.table("tiller.bronze.tb_despesas_bronze")

        # 🧩 Se o DataFrame estiver vazio, encerra
        if df_bronze.count() == 0:
            print("DataFrame Bronze vazio. Nenhum registro para processar.")
            return None

        # Transformações para Silver
        df_silver = (
            df_bronze
            .withColumn("data_emissao", F.to_date("datEmissao"))
            .withColumn("ano", F.col("numAno").cast("int"))
            .withColumn("mes", F.col("numMes").cast("int"))
            .withColumn("valor_documento", F.col("vlrDocumento").cast("double"))
            .withColumn("valor_glosa", F.col("vlrGlosa").cast("double"))
            .withColumn("valor_liquido", F.col("vlrLiquido").cast("double"))
            .withColumn("data_ingestao_silver", F.current_timestamp())
            
            # Tratar valores nulos
            .withColumn("txtFornecedor", F.when(F.col("txtFornecedor").isNull(), "Não informado").otherwise(F.col("txtFornecedor")))
            .withColumn("txtNumero", F.when(F.col("txtNumero").isNull(), "Não informado").otherwise(F.col("txtNumero")))
            .withColumn("txtDescricao", F.initcap(F.trim("txtDescricao")))
        )

        print("Schema do DataFrame Silver:")
        df_silver.printSchema()

        print("Amostra de registros Silver:")
        df_silver.show(5, truncate=False)

        # Salva a tabela Silver
        df_silver.write.format("delta").mode("overwrite").saveAsTable("tiller.silver.tb_despesas_silver")

        print("Tabela 'tb_despesas_silver' criada/atualizada com sucesso!")
        return df_silver

    except Exception as e:
        print(f"Erro ao processar despesas Silver: {e}")
        return None

# Executa o processamento
despesas_silver = process_despesas_silver()


In [0]:
%sql
select * from tiller.silver.tb_despesas_silver;